In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from pytorch_pretrained_bert.modeling import BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [4]:
# other parameters
#MAX_SEQUENCE_LENGTH = 554
EMBEDDING_DIM = 768 #1024
#nb_words = 3438
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

def get_bert_embed_matrix():
    bert = BertModel.from_pretrained(BERT_FP)
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


def get_bert_embed_matrix():
    bert = BertModel.from_pretrained('bert-base-uncased')
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


g_word_embedding_matrix = get_bert_embed_matrix()

# # load model
# EMBEDDING_DIM = 300
# file_loc = '../../data/glove.42B.300d.txt'
# print (file_loc)

# gembeddings_index = {}
# with codecs.open(file_loc, encoding='utf-8') as f:
#     for line in f:
#         values = line.split(' ')
#         word = values[0]
#         gembedding = np.asarray(values[1:], dtype='float32')
#         gembeddings_index[word] = gembedding
# #
# f.close()
# print('G Word embeddings:', len(gembeddings_index))

# nb_words = len(word_index) +1
# g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
# for word, i in word_index.items():
#     gembedding_vector = gembeddings_index.get(word)
#     if gembedding_vector is not None:
#         g_word_embedding_matrix[i] = gembedding_vector
        
# print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
# split = 7869
# earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
#                           restore_best_weights=True)

# # remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_text = x_train_text[mask]
# model: GRU


Found 3437 unique tokens


In [5]:
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(*g_word_embedding_matrix.shape,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    #net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    #out_text = Dropout(0.5)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
model = text_model1()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 768)     23440896    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 256)     918528      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 128)          164352      bidirectional_1[0][0]            
___________

In [6]:
# main function for for LOSO
def main(time):
    model = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_text[:7869], 
                      vad[:7869].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_text[7869:], vad[7869:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    weight = []
    for time in range(30):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('text_blstm_bert7869_no_dropout.csv', header=['time','v','a','d', 'ave'])



Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 123s 19ms/step - loss: 1.9703 - v_loss: 0.6485 - a_loss: 0.6683 - d_loss: 0.6530 - v_ccc: 0.3510 - a_ccc: 0.3315 - d_ccc: 0.3472 - val_loss: 1.9624 - val_v_loss: 0.6851 - val_a_loss: 0.6487 - val_d_loss: 0.6518 - val_v_ccc: 0.3202 - val_a_ccc: 0.3592 - val_d_ccc: 0.3581
Epoch 2/50
6295/6295 [==============================] - 117s 19ms/step - loss: 1.4275 - v_loss: 0.3935 - a_loss: 0.5269 - d_loss: 0.5071 - v_ccc: 0.6065 - a_ccc: 0.4729 - d_ccc: 0.4931 - val_loss: 1.9905 - val_v_loss: 0.6652 - val_a_loss: 0.6541 - val_d_loss: 0.6919 - val_v_ccc: 0.3390 - val_a_ccc: 0.3538 - val_d_ccc: 0.3167
Epoch 3/50
6295/6295 [==============================] - 116s 18ms/step - loss: 1.2640 - v_loss: 0.3349 - a_loss: 0.4726 - d_loss: 0.4564 - v_ccc: 0.6651 - a_ccc: 0.5272 - d_ccc: 0.5437 - val_loss: 1.9395 - val_v_loss: 0.6119 - val_a_loss: 0.6434 - val_d_loss: 0.7055 - val_v_ccc: 0.3930 - val_a_cc

Epoch 9/50
6295/6295 [==============================] - 117s 19ms/step - loss: 0.8246 - v_loss: 0.2064 - a_loss: 0.3131 - d_loss: 0.3056 - v_ccc: 0.7937 - a_ccc: 0.6872 - d_ccc: 0.6945 - val_loss: 1.8956 - val_v_loss: 0.5844 - val_a_loss: 0.6434 - val_d_loss: 0.6898 - val_v_ccc: 0.4218 - val_a_ccc: 0.3632 - val_d_ccc: 0.3194
Epoch 10/50
6295/6295 [==============================] - 117s 19ms/step - loss: 0.7883 - v_loss: 0.1968 - a_loss: 0.2969 - d_loss: 0.2944 - v_ccc: 0.8031 - a_ccc: 0.7031 - d_ccc: 0.7055 - val_loss: 1.9545 - val_v_loss: 0.5710 - val_a_loss: 0.6929 - val_d_loss: 0.7110 - val_v_ccc: 0.4345 - val_a_ccc: 0.3129 - val_d_ccc: 0.2981
Epoch 11/50
6295/6295 [==============================] - 117s 19ms/step - loss: 0.7460 - v_loss: 0.1871 - a_loss: 0.2825 - d_loss: 0.2769 - v_ccc: 0.8131 - a_ccc: 0.7178 - d_ccc: 0.7232 - val_loss: 1.9037 - val_v_loss: 0.5710 - val_a_loss: 0.6613 - val_d_loss: 0.6932 - val_v_ccc: 0.4360 - val_a_ccc: 0.3452 - val_d_ccc: 0.3151
Epoch 12/50
6295/

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 122s 19ms/step - loss: 0.9640 - v_loss: 0.2397 - a_loss: 0.3656 - d_loss: 0.3586 - v_ccc: 0.7603 - a_ccc: 0.6344 - d_ccc: 0.6412 - val_loss: 1.8757 - val_v_loss: 0.5775 - val_a_loss: 0.6385 - val_d_loss: 0.6799 - val_v_ccc: 0.4246 - val_a_ccc: 0.3700 - val_d_ccc: 0.3297
Epoch 7/50
6295/6295 [==============================] - 121s 19ms/step - loss: 0.9091 - v_loss: 0.2293 - a_loss: 0.3417 - d_loss: 0.3379 - v_ccc: 0.7706 - a_ccc: 0.6583 - d_ccc: 0.6619 - val_loss: 1.9109 - val_v_loss: 0.5644 - val_a_loss: 0.6923 - val_d_loss: 0.6737 - val_v_ccc: 0.4396 - val_a_ccc: 0.3139 - val_d_ccc: 0.3356
Epoch 8/50
6295/6295 [==============================] - 121s 19ms/step - loss: 0.8592 - v_loss: 0.2151 - a_loss: 0.3247 - d_loss: 0.3202 - v_ccc: 0.7849 - a_ccc: 0.6757 - d_ccc: 0.6802 - val_loss: 1.9009 - val_v_loss: 0.5709 - val_a_loss: 0.6520 - val_d_loss: 0.7004 - val_v_ccc: 0.4350 - val_a_ccc: 0.3546 - val_d_ccc: 0.3095
Epoch 9/50
6295/6295 [========

6295/6295 [==============================] - 122s 19ms/step - loss: 1.4187 - v_loss: 0.3867 - a_loss: 0.5218 - d_loss: 0.5103 - v_ccc: 0.6132 - a_ccc: 0.4784 - d_ccc: 0.4897 - val_loss: 1.9315 - val_v_loss: 0.6425 - val_a_loss: 0.6415 - val_d_loss: 0.6679 - val_v_ccc: 0.3611 - val_a_ccc: 0.3666 - val_d_ccc: 0.3407
Epoch 3/50
6295/6295 [==============================] - 122s 19ms/step - loss: 1.2650 - v_loss: 0.3277 - a_loss: 0.4734 - d_loss: 0.4638 - v_ccc: 0.6724 - a_ccc: 0.5265 - d_ccc: 0.5361 - val_loss: 1.9313 - val_v_loss: 0.6270 - val_a_loss: 0.6369 - val_d_loss: 0.6872 - val_v_ccc: 0.3751 - val_a_ccc: 0.3723 - val_d_ccc: 0.3213
Epoch 4/50
6295/6295 [==============================] - 121s 19ms/step - loss: 1.1336 - v_loss: 0.2847 - a_loss: 0.4277 - d_loss: 0.4213 - v_ccc: 0.7153 - a_ccc: 0.5723 - d_ccc: 0.5788 - val_loss: 1.9055 - val_v_loss: 0.6088 - val_a_loss: 0.6397 - val_d_loss: 0.6778 - val_v_ccc: 0.3953 - val_a_ccc: 0.3683 - val_d_ccc: 0.3309
Epoch 5/50
6295/6295 [========

...

6295/6295 [==============================] - 120s 19ms/step - loss: 0.8463 - v_loss: 0.2125 - a_loss: 0.3205 - d_loss: 0.3136 - v_ccc: 0.7876 - a_ccc: 0.6797 - d_ccc: 0.6864 - val_loss: 1.9133 - val_v_loss: 0.5803 - val_a_loss: 0.6606 - val_d_loss: 0.6940 - val_v_ccc: 0.4260 - val_a_ccc: 0.3462 - val_d_ccc: 0.3145
Epoch 9/50
6295/6295 [==============================] - 120s 19ms/step - loss: 0.8111 - v_loss: 0.2012 - a_loss: 0.3076 - d_loss: 0.3027 - v_ccc: 0.7989 - a_ccc: 0.6925 - d_ccc: 0.6975 - val_loss: 1.9497 - val_v_loss: 0.5717 - val_a_loss: 0.6742 - val_d_loss: 0.7233 - val_v_ccc: 0.4338 - val_a_ccc: 0.3315 - val_d_ccc: 0.2850
Epoch 10/50
6295/6295 [==============================] - 121s 19ms/step - loss: 0.7684 - v_loss: 0.1946 - a_loss: 0.2884 - d_loss: 0.2850 - v_ccc: 0.8053 - a_ccc: 0.7113 - d_ccc: 0.7150 - val_loss: 1.8706 - val_v_loss: 0.5611 - val_a_loss: 0.6567 - val_d_loss: 0.6754 - val_v_ccc: 0.4458 - val_a_ccc: 0.3498 - val_d_ccc: 0.3338
Epoch 11/50
6295/6295 [======

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 118s 19ms/step - loss: 0.7241 - v_loss: 0.1840 - a_loss: 0.2689 - d_loss: 0.2710 - v_ccc: 0.8160 - a_ccc: 0.7309 - d_ccc: 0.7290 - val_loss: 1.9385 - val_v_loss: 0.5697 - val_a_loss: 0.6835 - val_d_loss: 0.7051 - val_v_ccc: 0.4358 - val_a_ccc: 0.3224 - val_d_ccc: 0.3033
Epoch 12/50
6295/6295 [==============================] - 120s 19ms/step - loss: 0.6918 - v_loss: 0.1769 - a_loss: 0.2578 - d_loss: 0.2576 - v_ccc: 0.8232 - a_ccc: 0.7424 - d_ccc: 0.7426 - val_loss: 1.9692 - val_v_loss: 0.5648 - val_a_loss: 0.7123 - val_d_loss: 0.7128 - val_v_ccc: 0.4415 - val_a_ccc: 0.2930 - val_d_ccc: 0.2963
Epoch 13/50
6295/6295 [==============================] - 119s 19ms/step - loss: 0.6584 - v_loss: 0.1684 - a_loss: 0.2460 - d_loss: 0.2437 - v_ccc: 0.8316 - a_ccc: 0.7539 - d_ccc: 0.7561 - val_loss: 1.9366 - val_v_loss: 0.5717 - val_a_loss: 0.6785 - val_d_loss: 0.7083 - val_v_ccc: 0.4353 - val_a_ccc: 0.3275 - val_d_ccc: 0.3006
Epoch 14/50
6295/6295 [=====

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 122s 19ms/step - loss: 0.9706 - v_loss: 0.2429 - a_loss: 0.3652 - d_loss: 0.3633 - v_ccc: 0.7573 - a_ccc: 0.6351 - d_ccc: 0.6370 - val_loss: 1.8677 - val_v_loss: 0.5610 - val_a_loss: 0.6353 - val_d_loss: 0.6888 - val_v_ccc: 0.4400 - val_a_ccc: 0.3718 - val_d_ccc: 0.3205
Epoch 7/50
6295/6295 [==============================] - 122s 19ms/step - loss: 0.9080 - v_loss: 0.2284 - a_loss: 0.3422 - d_loss: 0.3377 - v_ccc: 0.7716 - a_ccc: 0.6580 - d_ccc: 0.6624 - val_loss: 1.8667 - val_v_loss: 0.5597 - val_a_loss: 0.6416 - val_d_loss: 0.6852 - val_v_ccc: 0.4429 - val_a_ccc: 0.3660 - val_d_ccc: 0.3245
Epoch 8/50
6295/6295 [==============================] - 122s 19ms/step - loss: 0.8487 - v_loss: 0.2137 - a_loss: 0.3164 - d_loss: 0.3187 - v_ccc: 0.7863 - a_ccc: 0.6835 - d_ccc: 0.6815 - val_loss: 1.9329 - val_v_loss: 0.5779 - val_a_loss: 0.6888 - val_d_loss: 0.6835 - val_v_ccc: 0.4252 - val_a_ccc: 0.3165 - val_d_ccc: 0.3253
Epoch 9/50
6295/6295 [========